# Single-cell ATACseq Tutorial

## Background
This tutorial demonstrates the principle behind the analysis of spatially resolvable single-cell ATACseq data. The transposase-accessible chromatin using sequencing (ATAC-Seq) assay has been widely adopted since its development by Jason Buenrostro and colleagues in the Greenleaf lab in 2013, and it’s now one of the most popular approaches to investigate chromatin openness and its dynamics.

ATAC-Seq is used to identify regions of the genome that have open chromatin states that are generally associated with sites undergoing active transcription. Transcription factor binding sites and positions of nucleosomes can also be identified from the analysis of ATAC-Seq data, potentially allowing important genetic pathways in the samples to be elucidated.

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S2001037020303019-ga1_lrg.jpg" alt="Drawing" style="width:1000px;"/>

## Overview and Purpose
The dataset we use here comes from Lareau et al., Nat Biotech 2019, one of the highest throughput single-cell ATAC-seq experiments to date. They assay 136K resting and stimulated bone marrow-derived cells and study the different cell types and lineages that emerge. The paper analyzes resting & stimulated cells separately (and we also find that these two populations split into largely distinct clusters), so here we only focus on the 60K resting cells from this experiment.

The cell metadata from this experiment includes annotated putative cell types based on chromVAR, Louvain clustering, and downstream cluster analysis. We compare our de novo identified clusters to these annotated clusters from the paper (and find good agreement).

The peak-cell matrix along with peak and cell metadata files are publicly available on GEO, GSE123580.

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41467-021-21583-9/MediaObjects/41467_2021_21583_Fig1_HTML.png?as=webp" alt="Drawing" style="width:1000px;"/>

## Standard single-cell ATACseq workflow
The steps below encompass the standard pre-processing workflow. These represent the selection and filtration of cells based on QC metrics, data normalization and scaling, and the detection of highly variable features.

This tutorial will cover the following tasks.
- QC and selecting cells
- Normalization
- Identifying highly variable features
- Scaling the data
- Linear Dimensional reduction 
- Determining Dimensionality
- Clustering
- Assigning cell type identity to clusters
- Detecting spatially variable features
- Interactive visualization
- Integration with single-cell RNA-seq data

In [ ]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/uuxpyhGNDsk?controls=0" frameborder="0" allowfullscreen></iframe>')

# Take a quiz

In [ ]:
!pip install jupyterquiz
import jupyterquiz
from jupyterquiz import display_quiz
display_quiz("/home/jupyter/tutorials/duplicateQuiz.json")

<div class="alert-info" style="font-size:200%">
STEP 1: Setting up environment
</div>

In [ ]:
!pip install ipywidgets

<div class="alert-info" style="font-size:200%">
STEP 1a: Run the following commands in TERMINAL
</div>

# The following commands (1b to 1d) should be performed in Terminal
To access terminal, follow this path File > New > Terminal

Then do Step 1b to 1d in Terminal

Do step 1e onwards in Jupyter notebook

# Step 1b: Create conda environment in terminal for rapids, then activate it 
conda create -n rapidsai

conda activate rapidsai

# Step 1c: install rapids
conda install -c rapidsai -c nvidia -c conda-forge python=3.7 cudatoolkit=11.2 rapids=21.12 llvmlite gcsfs openssl dask-sql
## install scanpy wget
pip install scanpy wget

# Step 1d: Add rapidsai environ
python -m ipykernel install --user --name=rapidsai

jupyter kernelspec list

## After completing step 1d in terminal, revert to Jupyter notebook to continue with the below tasks

# Step 1e download required files

In [ ]:
wget https://raw.githubusercontent.com/NVIDIA-Genomics-Research/rapids-single-cell-examples/master/notebooks/utils.py

In [ ]:
wget https://raw.githubusercontent.com/NVIDIA-Genomics-Research/rapids-single-cell-examples/master/notebooks/rapids_scanpy_funcs.py

In [ ]:
wget https://raw.githubusercontent.com/NVIDIA-Genomics-Research/rapids-single-cell-examples/master/notebooks/dsci_bmmc_60k_gpu.ipynb

# Step 1g open notebook and change kernel in jupyter notebook to use rapidsai envs
# jupyter menu: goto > kernel > change kernel > rapidsai

# Import Sample Files
Now lets create some folders to stay organized and copy over our prepared raw files. We're going to create a directory called "Tutorial1" which we'll use for this module. We'll then create subfolders for our InputFiles and for the files that we'll be creating during this module.

The following files will be downloaded when you execute the below command
- processed peak-cell count matrix for this dataset (.h5ad)
- set of nonzero peak names (.npy)
- cell metadata (.csv)
are stored inside the data folder

In [ ]:
#These commands create our directory structure.
!cd $HOMEDIR
!mkdir -p Tutorial2
!mkdir -p Tutorial2/InputFiles
!echo $PWD

#These commands help identify the google cloud storage bucket where the example files are held.
project_id = "nosi-unmc-seq"
original_bucket = "gs://unmc_singlecell_atac_data_examples"
#This command copies our example files to the Tutorial1/Inputfiles folder that we created above.
! gsutil cp $original_bucket/Tutorial1/dsci_resting_cell_metadata.csv Tutorial2/InputFiles
! gsutil cp $original_bucket/Tutorial1/dsci_resting_nonzeropeaks.h5ad Tutorial2/InputFiles
! gsutil cp $original_bucket/Tutorial1/dsci_resting_peaknames_nonzero.npy Tutorial2/InputFiles

# Let's make sure that the files copied correctly. 
# You should see 3 files after running the following command:

In [ ]:
!ls Tutorial2/InputFiles

# Installing required packages

In [ ]:
%pip install numpy
%pip install scanpy
%pip install anndata
%pip install matplotlib
%pip install utils
%pip install rapids

# *Restart Kernel here*
Do the following -> Menu tab > Kernel > Restart Kernel

In [ ]:
pip install scanpy wget

# *Restart Kernel here again*
Do the following -> Menu tab > Kernel > Restart Kernel

In [ ]:
import numpy as np
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
from collections import OrderedDict

import time
import os, wget

import cudf
import cupy as cp

from cuml.decomposition import PCA
from cuml.manifold import TSNE

import rapids_scanpy_funcs
import utils

import warnings
warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

# Replace and download supporting files

In [ ]:
!rm utils.py
!rm rapids_scanpy_funcs.py
!wget https://raw.githubusercontent.com/NVIDIA-Genomics-Research/rapids-single-cell-examples/master/notebooks/utils.py
!wget https://raw.githubusercontent.com/NVIDIA-Genomics-Research/rapids-single-cell-examples/master/notebooks/rapids_scanpy_funcs.py

In [13]:
import rmm

rmm.reinitialize(
    managed_memory=True, # Allows oversubscription
    devices=0, # GPU device IDs to register. By default registers only GPU 0.
)

cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

<div class="alert-info" style="font-size:200%">
STEP 2: Setting up parameters
</div>

In [72]:
# filtering peaks
n_top_peaks = 5000 # Number of peaks to retain

# PCA
n_components = 50 # Number of principal components to compute

# t-SNE
tsne_n_pcs = 20 # Number of principal components to use for t-SNE

# KNN
n_neighbors = 10 # Number of nearest neighbors for KNN graph
knn_n_pcs = 50 # Number of principal components to use for finding nearest neighbors

# UMAP
umap_min_dist = 0.4 
umap_spread = 1.0

# Differential peaks
n_diff_peaks = 50 # Number of differential peaks to compute for each cluster

# Marker genes
# Gene coordinates are in GRCh37, from grch37.ensembl.org
markers = ['MS4A1', 'LEF1', 'NKG7', 'TREM1', 'GATA1', 'PAX5']
marker_coords = OrderedDict()
marker_coords['MS4A1'] = ['chr11', 60223225, 60238233, 'fwd']
marker_coords['LEF1'] = ['chr4', 108968701, 109090112, 'rev']
marker_coords['NKG7'] = ['chr19', 51874860, 51875969, 'rev']
marker_coords['TREM1'] = ['chr6', 41235664, 41254457, 'rev']
marker_coords['GATA1'] = ['chrX', 48644962, 48652716, 'fwd']
marker_coords['PAX5'] = ['chr9', 36833272, 37034103, 'rev']

In [73]:
start_time = time.time()

In [ ]:
%%time
input_file = "../data/dsci_resting_nonzeropeaks.h5ad"

if not os.path.exists(input_file):
    print('Downloading import file...')
    os.makedirs('../data', exist_ok=True)
    wget.download('https://rapids-single-cell-examples.s3.us-east-2.amazonaws.com/dsci_resting_nonzeropeaks.h5ad',
                  input_file)

adata = anndata.read_h5ad(input_file)
adata_raw = adata.copy()
print(adata.X.shape)

In [75]:
preprocessing_start = time.time()

<div class="alert-info" style="font-size:200%">
STEP 3: Preprocessing
</div>

In [ ]:
%%time

lognormalized = utils.logtf_idf(adata.X, pseudocount=10**3)
adata.X = lognormalized

In [ ]:
%%time
adata = utils.filter_peaks(adata, n_top_peaks)
print(adata.X.shape)

In [ ]:
print("Preprocessing time: %.2fsec" % (time.time() - preprocessing_start))

<div class="alert-info" style="font-size:200%">
STEP 4: Clustering & Visualization
</div>

## PCA

In [ ]:
%%time
adata = anndata.AnnData(X=adata.X.todense(),
                        obs=adata.obs,
                        var=adata.var)
adata.obsm["X_pca"] = PCA(n_components=n_components).fit_transform(adata.X)
adata.obsm["X_pca"].shape

# UMAP visualization

In [ ]:
%%time
sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=knn_n_pcs, method='rapids')

In [ ]:
%%time
sc.tl.umap(adata, min_dist=umap_min_dist, spread=umap_spread, method='rapids')

# Graph clustering
Below, we show how to use the RAPIDS Louvain clustering or Leiden clustering functions to cluster the data.

In [ ]:
%%time
sc.tl.louvain(adata, flavor='rapids')

In [ ]:
%%time
adata.obs['leiden'] = rapids_scanpy_funcs.leiden(adata)

Below, we plot the UMAP visualization of the cells colored by the Louvain clusters. Next, we color the cells using the cell type annotations reported in the source paper. We see that the clusters we discovered match closely with the reported cell types.

In [ ]:
%%time

fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize = (12, 3), dpi=100)

sc.pl.umap(adata, color=['louvain'], ax=axs[0], show=False)
sc.pl.umap(adata, color=['leiden'], ax=axs[1], show=False)
sc.pl.umap(adata, color=['cell_type'], ax=axs[2], show=False)

plt.tight_layout()
plt.show()

## tSNE visualization

In [ ]:
%%time
adata.obsm['X_tsne'] = TSNE().fit_transform(adata.obsm["X_pca"][:, :tsne_n_pcs])

In [ ]:
%%time

fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize = (12, 3), dpi=100)

sc.pl.tsne(adata, color=['louvain'], ax=axs[0], show=False)
sc.pl.tsne(adata, color=['leiden'], ax=axs[1], show=False)
sc.pl.tsne(adata, color=['cell_type'], ax=axs[2], show=False)

plt.tight_layout()
plt.show()

## Marker genes

We now compute a gene activity score representing the activity of each marker gene in each cell. We use the adata_raw object to compute this score. This allows us to consider all peaks, not just the subset we used for clustering and UMAP.

In [ ]:
%%time
# find which peaks overlap with each marker gene (+ 5kb upstream)
peaks_per_gene = {gene : [] for gene in markers}

for (i, peak) in enumerate(adata_raw.var_names):
    chrom, coords = peak.strip().split(':')
    start, end = [int(c) for c in coords.split('-')]
    for gene in markers:
        if utils.overlap(marker_coords[gene], [chrom, start, end], 5000, 0):
            peaks_per_gene[gene].append((i, peak))

for gene in peaks_per_gene:
    print(f"{gene} has {len(peaks_per_gene[gene])} nearby peaks")

In [ ]:
%%time
# Calculate per-cell gene activity scores for each marker gene
cell_gene_scores = np.zeros((adata.X.shape[0], len(marker_coords.keys())))
for (i, gene) in enumerate(peaks_per_gene.keys()):
    indices = [x[0] for x in peaks_per_gene[gene]]
    peak_scores = adata_raw.X[:, indices].toarray()
    cell_gene_scores[:, i] = np.sum(peak_scores, axis=1) + 1

# Store per-cell gene activity scores with adata
for (i, gene) in enumerate(markers):
    adata.obs[gene] = np.log2(cell_gene_scores[:, i].tolist())
    adata.obs.loc[np.isneginf(adata.obs[gene]), gene] = 0

We now visualize the UMAP plot colored by activity of marker genes. We observe that the activation of cell-type specific marker genes correlates well with the cell types identified by Louvain clustering.

In [ ]:
%%time

fig, axs = plt.subplots(nrows=2, ncols=3, figsize = (8, 4), dpi=100)
axs = axs.flatten()
for (i, gene) in enumerate(markers):
    sc.pl.umap(adata, color_map="Blues", color=gene, ax = axs[i], show=False)
plt.tight_layout()
plt.show()

<div class="alert-info" style="font-size:200%">
STEP 5: Finding differential peaks
</div>

## Find Differential peaks
We perform an accelerated logistic regression-based differential peak computation using RAPIDS. This function may not give the exact same results as the equivalent Scanpy CPU function.

In [ ]:
%%time
cluster_labels = cudf.Series.from_categorical(adata.obs["louvain"].cat)
var_names = cudf.Series(adata.var_names)
dense_gpu_array = cp.sparse.csr_matrix(cp.array(adata.X))

scores, names, reference = rapids_scanpy_funcs.rank_genes_groups(
    dense_gpu_array,
    cluster_labels, 
    var_names, 
    penalty='none',
    n_genes=n_diff_peaks, groups='all', reference='rest')

In [ ]:
%%time
fig, axs = plt.subplots(len(names[0])//3 + 1, 3, figsize=(11,11), dpi=100)
axs = axs.flatten()

for (i, peak) in enumerate(names[0]):
    sc.pl.umap(adata, color=peak, 
           ax=axs[i], show=False,
           vmax=10, vmin=-0.5,
           cmap='Blues'
          )
plt.tight_layout()
plt.show()

In [ ]:
print("Full time: %.2fsec" % (time.time() - start_time))